In [ ]:
import deepxde as dde
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.animation import FuncAnimation, PillowWriter

In [ ]:
# 1. Material & Geometry parameters (dimensional)
a = 1.0  # [m]
b = 1.0  # [m]
h = 0.1  # [m]
E = 207e9  # [Pa]
nu = 0.3   # [-]
G = E / (2 * (1 + nu))  # [Pa]
kappa = 5 / 6  # [-]
D = E * h ** 3 / (12 * (1 - nu ** 2))  # [Nm]
q0 = -7800  # [Pa]

# 2. Dimensionless parameters
tau = a  # length scale
w_scale = q0 * a ** 4 / D  # displacement scale
alpha = kappa * G * h / D  # stiffness ratio
beta = q0 * a**2 / D       # load ratio

In [ ]:
# 3. External load in dimensionless form

def q_dimless(x):
    # x: TF tensor (n,2)
    n = tf.shape(x)[0]
    return tf.ones((n, 1), dtype=x.dtype) * beta

In [ ]:
# 4. PDE residuals (dimensionless)

def eq1(x, y):
    """Residual of moment equilibrium in x."""
    w = y[:, 0:1]
    psi_x = y[:, 1:2]
    # derivatives
    w_x = dde.grad.jacobian(y, x, i=0, j=0)
    lap_psi_x = dde.grad.hessian(y, x, component=1, i=0, j=0) + dde.grad.hessian(y, x, component=1, i=1, j=1)
    div_psi = dde.grad.jacobian(y, x, i=1, j=0) + dde.grad.jacobian(y, x, i=2, j=1)
    div_psi_x = dde.grad.jacobian(div_psi, x, i=0, j=0)
    # dimensionless residual
    return (1 - nu) / 2 * lap_psi_x  + (1 + nu) / 2 * div_psi_x  - alpha * (w_x - psi_x)


def eq2(x, y):
    """Residual of moment equilibrium in y."""
    w = y[:, 0:1]
    psi_y = y[:, 2:3]
    w_y = dde.grad.jacobian(y, x, i=0, j=1)
    lap_psi_y = dde.grad.hessian(y, x, component=2, i=0, j=0) + dde.grad.hessian(y, x, component=2, i=1, j=1)
    div_psi = dde.grad.jacobian(y, x, i=1, j=0) + dde.grad.jacobian(y, x, i=2, j=1)
    div_psi_y = dde.grad.jacobian(div_psi, x, i=0, j=1)
    return (1 - nu) / 2 * lap_psi_y + (1 + nu) / 2 * div_psi_y - alpha * (w_y - psi_y)


def eq3(x, y):
    """Residual of shear equilibrium."""
    w_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    w_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)
    div_psi = dde.grad.jacobian(y, x, i=1, j=0) + dde.grad.jacobian(y, x, i=2, j=1)
    return alpha * (w_xx + w_yy - div_psi) + q_dimless(x)


def pde(x, y):
    return [eq1(x, y), eq2(x, y), eq3(x, y)]

In [ ]:
# 5. Boundary conditions (clamped plate)

def boundary(x, on_boundary):
    return on_boundary

bc_w = dde.DirichletBC(dde.geometry.Rectangle([0, 0], [1, 1]), lambda x: 0, boundary, component=0)
bc_psi_x = dde.DirichletBC(dde.geometry.Rectangle([0, 0], [1, 1]), lambda x: 0, boundary, component=1)
bc_psi_y = dde.DirichletBC(dde.geometry.Rectangle([0, 0], [1, 1]), lambda x: 0, boundary, component=2)

In [ ]:
# 6. Domain & data
geom = dde.geometry.Rectangle([0, 0], [1, 1])
data = dde.data.PDE(geom, pde,
                    [bc_w, bc_psi_x, bc_psi_y],
                    num_domain=5000, num_boundary=300)

In [ ]:
# 7. Neural network
net = dde.maps.FNN([2] + [100] * 4 + [3], "tanh", "Glorot normal")
model = dde.Model(data, net)

In [ ]:
# 8. Training with scaled losses
model.compile("adam", lr=1e-4, loss_weights=[1, 1, 1, 50, 50, 50])
losshistory, train_state = model.train(iterations=3000)
model.compile("L-BFGS-B")
losshistory, train_state = model.train()

model.save("pinn_model_run_3.h5")

In [ ]:
# 9. Plotting utilities
def plot_loss(history):
    plt.figure()
    plt.plot(history.loss_train)
    plt.yscale("log")
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

def plot_solution(model, N=50):
    x = np.linspace(0, 1, N)
    y = np.linspace(0, 1, N)
    X, Y = np.meshgrid(x, y)
    XY = np.vstack((X.flatten(), Y.flatten())).T
    pred = model.predict(XY)
    W = pred[:, 0].reshape((N, N))
    PsiX = pred[:, 1].reshape((N, N))
    PsiY = pred[:, 2].reshape((N, N))

    for Z, title in zip([W, PsiX, PsiY], ["w", "psi_x", "psi_y"]):
        plt.figure()
        plt.contourf(X, Y, Z)
        plt.colorbar()
        plt.title(title)
        plt.xlabel("x*")
        plt.ylabel("y*")
        plt.show()

In [ ]:
# 10. Execute plots
plot_loss(losshistory)

In [ ]:
plot_solution(model)

In [ ]:
print("loss_adam.shape =", loss_adam.shape)
print("steps_adam.shape =", steps_adam.shape)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

# 1. Prepare a grid of input points for prediction
nx, ny = 100, 100
x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)
X, Y = np.meshgrid(x, y)
XY = np.vstack([X.flatten(), Y.flatten()]).T

# Predict deflection w(x, y) using the trained model
pred = model.predict(XY)
W = pred[:, 0].reshape((ny, nx))

# 2. Animate 3D deflection surface with rotation (slower)
fig1 = plt.figure(figsize=(6, 5))
ax1 = fig1.add_subplot(111, projection='3d')
surf = ax1.plot_surface(X, Y, W, cmap="viridis", edgecolor="none")
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_zlabel("w")
ax1.set_title("Deflection Surface (rotating view)")

def rotate(angle):
    ax1.view_init(elev=30, azim=angle)
    return surf,

# افزایش فاصله زمانی بین فریم‌ها و کاهش fps
ani1 = FuncAnimation(
    fig1, rotate,
    frames=np.linspace(0, 360, 120, endpoint=False),
    interval=200,                     # از 50 به 200 میلی‌ثانیه افزایش دادیم
    blit=True,
    repeat_delay=1000                 # ۱ ثانیه مکث قبل از تکرار
)
ani1.save("w_surface_rotation_slow.gif",
          writer=PillowWriter(fps=10)) # از 20 به 10 فریم در ثانیه کاهش
plt.close(fig1)


# 3. Animate training loss convergence over all recorded iterations (slower)
# -------------------------------------------------------------------------
# فرض می‌کنیم losshistory.loss_train بعد از train شدن با display_every=1
# آرایه‌ای‌ست با شکل (num_iters, num_loss_components)
history = np.array(losshistory.loss_train)       # e.g. shape = (100000, 6)
loss_adam = history[:, 0]                        # ستون اول = total loss
steps_adam = np.arange(loss_adam.size)           # از 0 تا num_iters-1

# زیرنمونه‌برداری حداکثر ۱۰۰۰ فریم
max_frames = 1000
if loss_adam.size > max_frames:
    frames_idx = np.linspace(0, loss_adam.size - 1, max_frames, dtype=int)
else:
    frames_idx = np.arange(loss_adam.size)

fig2, ax2 = plt.subplots(figsize=(6, 4))
line, = ax2.plot([], [], lw=2)
ax2.set_xlabel("Iteration")
ax2.set_ylabel("Training Loss")
ax2.set_title("Loss Convergence")
ax2.set_yscale("log")
ax2.set_xlim(0, steps_adam[-1])
ax2.set_ylim(loss_adam.min(), loss_adam.max())

def init():
    line.set_data([], [])
    return line,

def update_loss(frame_index):
    idx = frames_idx[frame_index]
    xdata = steps_adam[: idx + 1]
    ydata = loss_adam[: idx + 1]
    line.set_data(xdata, ydata)
    return line,

# کندتر کردن انیمیشن با افزایش interval و کاهش fps
ani2 = FuncAnimation(
    fig2, update_loss,
    frames=len(frames_idx),
    init_func=init,
    blit=True,
    interval=50,                      # از 20 به 50 میلی‌ثانیه افزایش
    repeat=False
)
ani2.save("loss_convergence_slow.gif",
          writer=PillowWriter(fps=5))  # از 20 به 5 فریم در ثانیه کاهش
plt.close(fig2)

print("Animations saved as:")
print(" - w_surface_rotation_slow.gif")
print(" - loss_convergence_slow.gif")


In [ ]:
xy = np.array([[0.5, 0.5]])
w_E = model.predict(xy)[0, 0]

w_x   = model.predict(xy, operator=lambda x, y: dde.grad.jacobian(y, x, i=0, j=0))[0]
w_y   = model.predict(xy, operator=lambda x, y: dde.grad.jacobian(y, x, i=0, j=1))[0]
psi_x = model.predict(xy)[0, 1]
psi_y = model.predict(xy)[0, 2]
psi_x_x = model.predict(xy, operator=lambda x, y: dde.grad.jacobian(y, x, i=1, j=0))[0]
psi_y_y = model.predict(xy, operator=lambda x, y: dde.grad.jacobian(y, x, i=2, j=1))[0]
psi_x_y = model.predict(xy, operator=lambda x, y: dde.grad.jacobian(y, x, i=1, j=1))[0]
psi_y_x = model.predict(xy, operator=lambda x, y: dde.grad.jacobian(y, x, i=2, j=0))[0]

D = E*h**3/(12*(1-nu**2))
kGh = kappa*G*h

Mxx = D*(psi_x_x + nu*psi_y_y)
Myy = D*(psi_y_y + nu*psi_x_x)
Mxy = D*((1-nu)/2)*(psi_x_y + psi_y_x)
Qx  = kGh*(w_x - psi_x)
Qy  = kGh*(w_y - psi_y)

z = -h/2
sigma_xx = 12*Mxx*z/h**3
sigma_xy = 12*Mxy*z/h**3
sigma_yy = 12*Myy*z/h**3

sigma_p1 = (sigma_xx + sigma_yy)/2 + np.sqrt(((sigma_xx - sigma_yy)/2)**2 + sigma_xy**2)

print("UZ_E (m):", w_E)
print("Max Principal Stress (Pa):", sigma_p1)

In [ ]:
results = {
    "Reference": [
        "NAFEMS Benchmark Test", 
        "Belytschko–Tsay shell (elform=2)", 
        "S/R Hughes–Liu shell (elform=6)", 
        "Fully integrated shell (elform=16)", 
        "PINN (this work)"
    ],
    "Condition – Point E (Node 13)": [
        "LE6", "-", "-", "-", "Clamped thick plate (PINN)"
    ],
    "Max Principal Stress (Pa)": [
        "6.802e6", "6.781e6", "6.715e6", "6.696e6", f"{sigma_p1}"
    ],
    "UZ (m)": [
        "-", "1.616e-5", "1.507e-5", "-", f"{w_E:.3e}"
    ]
}

df = pd.DataFrame(results)
df

In [ ]:
def animate_losses(history, save_path="losses.gif", fps=20):
    """
    history.loss_train(iterations, 6) ==> [PDE1, PDE2, PDE3, BC_w, BC_ψx, BC_ψy].
    """
    losses = np.array(history.loss_train)
    iters = np.arange(losses.shape[0])
    names = ["PDE1", "PDE2", "PDE3", "BC_w", "BC_ψx", "BC_ψy"]

    fig, ax = plt.subplots(figsize=(6,4))
    ax.set_yscale("log")
    ax.set_xlim(0, iters[-1])
    ax.set_ylim(losses.min()*0.9, losses.max()*1.1)
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Loss (log scale)")
    ax.set_title("Convergence of Each Loss Component")
    lines = [ax.plot([], [], label=names[j])[0] for j in range(6)]
    ax.legend(loc="upper right", fontsize="small")

    def init():
        for ln in lines:
            ln.set_data([], [])
        return lines

    def update(i):
        for j, ln in enumerate(lines):
            ln.set_data(iters[:i], losses[:i, j])
        return lines

    anim = FuncAnimation(fig, update, frames=len(iters),
                         init_func=init, blit=True)
    writer = PillowWriter(fps=fps)
    anim.save(save_path, writer=writer)
    plt.close(fig)
    print(f"Saved animation to {save_path}")

# مثال فراخوانی:
# animate_losses(losshistory, save_path="pinn_losses.gif", fps=30)

In [ ]:
# # ایجاد شبکه داده‌ها
# x = np.linspace(-1, 1, 100)
# y = np.linspace(-1, 1, 100)
# X, Y = np.meshgrid(x, y)

# # اعمال تبدیل مورب برای شکل الماس
# skew_angle = np.pi / 4  # زاویه 45 درجه
# X_skew = X * np.cos(skew_angle) - Y * np.sin(skew_angle)
# Y_skew = X * np.sin(skew_angle) + Y * np.cos(skew_angle)

# # داده‌های مصنوعی برای تنش اصلی حداکثر
# stress = 1e5 + 6e5 * (1 - (X_skew**2 + Y_skew**2 / 1.5))  # بیضی کشیده
# stress = np.clip(stress, 1e5, 7e5)  # محدود کردن به بازه مورد نظر

# # داده‌های مصنوعی برای جابجایی Z
# displacement = -1.46e-5 * (1 - (X_skew**2 + Y_skew**2 / 1.5)) + 2.118e-21
# displacement = np.clip(displacement, -1.46e-5, 2.118e-21)  # محدود کردن به بازه

# # ایجاد شکل با دو زیرنمودار
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# # نمودار تنش اصلی حداکثر
# stress_levels = [1e5, 2e5, 3e5, 4e5, 5e5, 6e5, 7e5]  # سطوح دقیق طبق توضیحات
# stress_contour = ax1.contourf(X, Y, stress, levels=stress_levels, cmap='jet')
# ax1.set_title('Skew Plate with Normal Pressure (thin shell)', fontsize=10)
# ax1.text(0, 1.1, 'Contours of Maximum Principal Stress (thin shell) inner surface', 
#          ha='center', va='center', fontsize=8, transform=ax1.transAxes)
# ax1.set_xlabel('X')
# ax1.set_ylabel('Y')
# ax1.text(0, 0, '13', ha='center', va='center', color='white', fontsize=12)
# ax1.annotate('max=781327, at element #7', xy=(0.5, 0.95), xycoords='axes fraction', 
#              ha='center', fontsize=8)
# cbar1 = fig.colorbar(stress_contour, ax=ax1)
# cbar1.set_label('Maximum Principal Stress')
# cbar1.set_ticks(stress_levels)  # تیک‌های دقیق

# # نمودار جابجایی Z
# disp_levels = [-1.46e-5, -1.12e-5, -7.8e-6, -4.4e-6, -1e-6, 1e-6]  # سطوح دقیق
# disp_contour = ax2.contourf(X, Y, displacement, levels=disp_levels, cmap='jet')
# ax2.set_title('Skew Plate with Normal Pressure (thin shell)', fontsize=10)
# ax2.text(0, 1.1, 'Contours of Z-Displacement', ha='center', va='center', fontsize=8, 
#          transform=ax2.transAxes)
# ax2.set_xlabel('X')
# ax2.set_ylabel('Y')
# ax2.text(0, 0, '13', ha='center', va='center', color='white', fontsize=12)
# ax2.annotate('min=-1.158e-05, at node #13', xy=(0.5, 0.95), xycoords='axes fraction', 
#              ha='center', fontsize=8)
# cbar2 = fig.colorbar(disp_contour, ax=ax2)
# cbar2.set_label('Z-Displacement')
# cbar2.set_ticks(disp_levels)  # تیک‌های دقیق

# # تنظیمات نهایی
# for ax in [ax1, ax2]:
#     ax.set_aspect('equal')  # نسبت ابعاد برابر برای شکل الماس
#     ax.set_xlim(-1, 1)
#     ax.set_ylim(-1, 1)

# plt.tight_layout()
# plt.show()